# Phising email detection and reply system powered by LLMs
### Autors: Aleksander Folfas, Hong Jing

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
from tensorflow.keras.models import load_model

In [2]:
print(tf.__version__)

print(hub.__version__)

2.10.1
0.16.1


## Initialization

In [3]:
%run ReplyGeneration.ipynb # running this notebook will download Mistral LLM if its not installed (5GB)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

File already exists.


llm_load_tensors:        CPU buffer size =  4892.99 MiB
...................................................................................................
llama_new_context_with_model: n_ctx      = 256
llama_new_context_with_model: n_batch    = 126
llama_new_context_with_model: n_ubatch   = 126
llama_new_context_with_model: freq_base  = 1000000.0
llama_new_context_with_model: freq_scale = 1
llama_kv_cache_init:        CPU KV buffer size =    32.00 MiB
llama_new_context_with_model: KV self size  =   32.00 MiB, K (f16):   16.00 MiB, V (f16):   16.00 MiB
llama_new_context_with_model:        CPU  output buffer size =     0.12 MiB
llama_new_context_with_model:        CPU compute buffer size =    19.81 MiB
llama_new_context_with_model: graph nodes  = 1030
llama_new_context_with_model: graph splits = 1
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0

## Input

In [4]:
# Process email in json file
sender, reply_subject, email = input_email_processing("email.json")

## Classification

import the classification model

In [5]:
# Define the custom objects
custom_objects = {'KerasLayer': hub.KerasLayer}

# Load the model with custom objects
model = load_model('email_classification_model.h5', custom_objects=custom_objects)

In [6]:
isPhishing = False
prediction = model.predict([email])
print(prediction)
if (prediction[0] > 0.95):
    isPhishing = True

1/1 [==============================] - 1s 1s/step
[[0.9966365]]


## Reply Generation

In [7]:
if isPhishing:
    template = generate_prompt_from_template(email)
    reply = generate_reply(template)


llama_print_timings:        load time =    9835.70 ms
llama_print_timings:      sample time =      12.98 ms /    77 runs   (    0.17 ms per token,  5930.83 tokens per second)
llama_print_timings: prompt eval time =   12420.61 ms /   158 tokens (   78.61 ms per token,    12.72 tokens per second)
llama_print_timings:        eval time =   11455.56 ms /    76 runs   (  150.73 ms per token,     6.63 tokens per second)
llama_print_timings:       total time =   24088.67 ms /   234 tokens


## Output

In [8]:
if isPhishing:
    output_reply_processing(sender, reply_subject, reply)
    print("Reply email created")
else:
    print("Email is not phishing")

Reply email created
